In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [3]:
import mysql.connector
from mysql.connector import Error

try:
    connection = mysql.connector.connect(host='db',
                                         database='test',
                                         user='test',
                                         password='test')

    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
except Error as e:
    print("Error while connecting to MySQL", e)

Connected to MySQL Server version  5.5.5-10.2.27-MariaDB-1:10.2.27+maria~bionic


In [4]:
mycursor = connection.cursor()
mycursor.execute("SHOW DATABASES")
myresult = mycursor.fetchall()
for x in myresult:
  print(x)

('information_schema',)
('test',)


In [5]:
mycursor = connection.cursor()
mycursor.execute("SHOW TABLES FROM test")
myresult = mycursor.fetchall()
for x in myresult:
  print(x)

('bonds_liquidity_metrics',)


In [10]:
mycursor = connection.cursor()

mycursor.execute("SELECT * FROM test.bonds_liquidity_metrics")

myresult = mycursor.fetchall()

i = 0
for x in myresult:
  print(x)
  i += 1
  if i > 10:
    break;

(238, 'XS0088543193', datetime.datetime(1998, 6, 24, 0, 0), datetime.datetime(2028, 6, 24, 0, 0), 2500000000.0, 0.002934100111495804, 0.8047337278106509, datetime.datetime(2019, 10, 5, 23, 58, 15))
(242, 'XS0114288789', datetime.datetime(2006, 11, 28, 0, 0), datetime.datetime(2030, 3, 31, 0, 0), 2972943262.835, 0.00575756233668458, 0.5295857988165681, datetime.datetime(2019, 10, 5, 23, 58, 15))
(2307, 'XS0191754729', datetime.datetime(2004, 4, 28, 0, 0), datetime.datetime(2034, 4, 28, 0, 0), 1200000000.0, 0.006005967690544801, 0.5059171597633136, datetime.datetime(2019, 10, 5, 23, 58, 15))
(4414, 'XS0220743776', datetime.datetime(2005, 6, 3, 0, 0), datetime.datetime(2020, 6, 3, 0, 0), 100000000.0, 0.013733866560732624, 0.22485207100591717, datetime.datetime(2019, 10, 5, 23, 58, 15))
(4546, 'XS0223715920', datetime.datetime(2005, 6, 27, 0, 0), datetime.datetime(2035, 6, 30, 0, 0), 22513000.0, 0.011914459285721282, 0.2455621301775148, datetime.datetime(2019, 10, 5, 23, 58, 15))
(5579, 'X

In [6]:
mycursor.close()
connection.close()

In [12]:
pip install fastparquet

     |████████████████████████████████| 153kB 1.0MB/s eta 0:00:01
     |████████████████████████████████| 61kB 13.8MB/s eta 0:00:01
  Created wheel for fastparquet: filename=fastparquet-0.3.2-cp37-cp37m-linux_x86_64.whl size=260841 sha256=b9ffd1b257fb2d9bec0c90db98b4d4e4a27c2e9cfd89d8b01098372621457a45
  Stored in directory: /home/jovyan/.cache/pip/wheels/b9/36/13/01416a760ddcab0eb8281ec9c9ffcbed945c9b831647c8b904
  Created wheel for thrift: filename=thrift-0.11.0-cp37-cp37m-linux_x86_64.whl size=390565 sha256=41dcba72e0fae67c0fd98cca0ffc5d23870dafb13012565230188c5a7e0be1d4
  Stored in directory: /home/jovyan/.cache/pip/wheels/be/36/81/0f93ba89a1cb7887c91937948519840a72c0ffdd57cac0ae8f
Successfully built fastparquet thrift
Note: you may need to restart the kernel to use updated packages.


In [13]:
# Поменять на то, что выдаст на консоль analytics-consumer_1 (или на текущую дату)
outputFolder = "/shara/liquidity/parquet_20191005/" # сюда сохранены метрики ликвидности в паркете
outputFolderCsv = "/shara/liquidity/csv_20191005/" # сюда сохранены метрики ликвидности в csv
outputFolderDelta = "/shara/liquidity/delta_20191005/" # сюда будет сохраняться поток котировок в дельте

In [20]:
#import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
            .master('local') \
            .appName('myAppName') \
            .config('spark.executor.memory', '1gb') \
            .config('spark.cores.max', '2') \
            .getOrCreate()

sc = spark.sparkContext

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

df = sqlContext.read.parquet(outputFolder)

pandas_df = df.toPandas()
pandas_df.head(10)

,id,isin,date_of_end_placing,maturity_date,usd_volume,bid_ask_spread_relative_median,bid_ask_spread_relative_rank
0,238,XS0088543193,1998-06-24,2028-06-24,2.500000e+09,0.002934,0.804734
1,242,XS0114288789,2006-11-28,2030-03-31,2.972943e+09,0.005758,0.529586
2,2307,XS0191754729,2004-04-28,2034-04-28,1.200000e+09,0.006006,0.505917
3,4414,XS0220743776,2005-06-03,2020-06-03,1.000000e+08,0.013734,0.224852
4,4546,XS0223715920,2005-06-27,2035-06-30,2.251300e+07,0.011914,0.245562
5,5579,XS0248160102,2006-03-15,2026-03-23,1.000660e+08,0.021978,0.147929
6,5586,XS0245586903,2006-02-27,NaT,8.084600e+07,0.006171,0.494083
7,6627,XS0274618247,2006-11-03,NaT,1.000000e+08,0.008971,0.340237
8,7071,XS0290580595,2007-03-01,2022-03-07,1.300000e+09,0.004315,0.668639
9,7393,XS0304274599,2007-06-01,2022-06-07,5.000000e+08,0.003892,0.730769


In [34]:
from pyspark.sql.functions import col
delta_df = sqlContext.read.parquet(outputFolderDelta)
#delta_df.sort(col("update_ts").desc())
pandas_delta_df = delta_df.toPandas()
pandas_delta_df.dtypes

bond                            int64
isin                           object
tg                              int64
date                           object
volume_money                  float64
bid                           float64
ask                           float64
update_ts              datetime64[ns]
baspread_rel_median           float64
baspread_rel_rank             float64
cnt                             int64
dtype: object

In [38]:
df_sorted = pandas_delta_df.sort_values(by="update_ts", ascending=False)
df_sorted.head(10)

,bond,isin,tg,date,volume_money,bid,ask,update_ts,baspread_rel_median,baspread_rel_rank,cnt
25110,33651,XS0867478124,237,2019-10-04,0.0,100.37,101.34,2019-10-04 18:18:58,0.011778,0.254438,137
49409,32983,XS0861981180,237,2019-10-04,0.0,102.52,102.99,2019-10-04 18:18:56,0.004283,0.674556,137
30674,31933,XS0848137708,237,2019-10-04,0.0,102.42,104.07,2019-10-04 18:18:50,0.007236,0.411243,137
12024,31789,XS0848530977,237,2019-10-04,0.0,104.46,105.03,2019-10-04 18:18:50,0.002479,0.843195,137
51312,16211,XS0617134092,237,2019-10-04,0.0,105.42,105.70,2019-10-04 18:18:36,0.011812,0.251479,137
21789,29503,XS0805570354,237,2019-10-04,0.0,105.75,106.03,2019-10-04 18:18:28,0.004147,0.695266,137
34552,29977,XS0810596832,237,2019-10-04,0.0,108.33,108.84,2019-10-04 18:18:26,0.007496,0.387574,137
49402,12439,XS0504954347,237,2019-10-04,0.0,101.50,101.82,2019-10-04 18:18:08,0.003533,0.751479,137
3955,11956,XS0484209159,237,2019-10-04,0.0,101.41,101.74,2019-10-04 18:18:03,0.003342,0.769231,137
5404,13690,XS0559915961,237,2019-10-04,0.0,115.48,116.62,2019-10-04 18:17:55,0.006202,0.491124,137


In [40]:
print(delta_df.count())

118035
